<a href="https://colab.research.google.com/github/jawaharganesh24189/DLA/blob/main/DLA_GAN_Dialogue_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🎭 GAN-based Dialogue Generation SystemThis notebook implements a **Generative Adversarial Network (GAN)** for generating contextual dialogues with character awareness.## 📋 Workflow Overview1. **Setup & Dependencies** - Install and import required libraries2. **Google Drive & Data Loading** - Mount drive and load dialogue datasets3. **Dialogue Parsing** - Extract characters and dialogues from text files4. **Data Processing & Tokenization** - Create vocabulary and training sequences5. **Model Architecture** - Define Generator and Discriminator networks6. **Model Training** - Train the GAN with adversarial loss7. **Dialogue Generation** - Generate new dialogues based on learned patterns8. **Evaluation & Persistence** - Evaluate results and save models---

## 📦 Section 1: Setup & DependenciesInstalling and importing all required libraries for the GAN dialogue system.

In [ ]:
# Install required packages (run this in Google Colab)# Uncomment if running for the first time# !pip install tensorflow numpy matplotlib

In [ ]:
# Import core dependenciesimport osimport reimport globimport jsonimport csvimport picklefrom dataclasses import dataclassfrom typing import List, Dict, Optional, Tuplefrom collections import defaultdict, Counterfrom io import StringIO# Import numerical and ML librariesimport numpy as npimport tensorflow as tffrom tensorflow import kerasfrom tensorflow.keras.models import Modelfrom tensorflow.keras.layers import (    Input, Dense, LSTM, Embedding, Dropout,     Concatenate, Bidirectional, LayerNormalization)from tensorflow.keras.preprocessing.text import Tokenizerfrom tensorflow.keras.preprocessing.sequence import pad_sequencesfrom tensorflow.keras.optimizers import Adamfrom tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping# Import visualization librariesimport matplotlib.pyplot as plt# Set random seeds for reproducibilitynp.random.seed(42)tf.random.set_seed(42)print("✅ All dependencies imported successfully!")print(f"TensorFlow version: {tf.__version__}")

## 💾 Section 2: Google Drive & Data LoadingMount Google Drive and configure dataset paths.

In [ ]:
# Mount Google Drive (only needed in Google Colab)try:    from google.colab import drive    drive.mount('/content/drive')    print("✅ Google Drive mounted successfully!")except:    print("⚠️  Not running in Colab - skipping drive mount")

In [ ]:
# Configure dataset path# Update this path to point to your dialogue dataset folderDATASET_PATH = '/content/drive/MyDrive/DLA_Notebooks_Data_PGPM/Dataset/'# Verify the path existsif os.path.exists(DATASET_PATH):    files = glob.glob(os.path.join(DATASET_PATH, '*.txt'))    print(f"✅ Dataset path found: {DATASET_PATH}")    print(f"📁 Number of .txt files: {len(files)}")else:    print(f"⚠️  Dataset path not found: {DATASET_PATH}")    print("Please update DATASET_PATH to point to your dialogue files")

## 🎬 Section 3: Dialogue Parsing & Character ExtractionParse multiple dialogue files and extract character information.

In [ ]:
# ============================================================================# DialogueParser Class# ============================================================================# This class handles parsing of dialogue files in various formats:# - Format 1: "context: ... response: ..." # - Format 2: "Character: dialogue text"# # It automatically creates cleandata.txt in "Character: dialogue" format# ============================================================================@dataclassclass DialogueTurn:    """Represents a single dialogue turn with context and response"""    context: str    response: str    metadata: Optional[Dict] = Noneclass DialogueParser:    """    Multi-format dialogue parser that extracts dialogues from text files.    Automatically creates cleandata.txt for downstream processing.    """        def __init__(self):        # Regex patterns for different dialogue formats        self.context_response_pattern = re.compile(            r'context:\s*(.+?)\s*response:\s*(.+?)(?=\ncontext:|$)',             re.DOTALL | re.IGNORECASE        )        self.dialogue_pattern = re.compile(            r'^(.+?):\s*(.+?)$',            re.MULTILINE        )        def parse_file(self, filepath: str) -> List[DialogueTurn]:        """Parse a single dialogue file"""        with open(filepath, 'r', encoding='utf-8', errors='ignore') as f:            content = f.read()                # Try context-response format first        turns = self._parse_context_response(content)                # Fall back to dialogue format if no matches        if not turns:            turns = self._parse_dialogue_format(content)                return turns        def _parse_context_response(self, content: str) -> List[DialogueTurn]:        """Parse 'context: ... response: ...' format"""        matches = self.context_response_pattern.findall(content)        turns = []                for context, response in matches:            context = self._clean_text(context)            response = self._clean_text(response)                        if context and response:                turns.append(DialogueTurn(                    context=context,                    response=response,                    metadata={"format": "context_response"}                ))                return turns        def _parse_dialogue_format(self, content: str) -> List[DialogueTurn]:        """Parse 'Character: dialogue' format"""        lines = content.strip().split('\n')        turns = []        context_buffer = []                for line in lines:            line = line.strip()            if not line:                continue                        match = self.dialogue_pattern.match(line)            if match:                speaker, dialogue = match.groups()                dialogue = self._clean_text(dialogue)                                if dialogue:                    # Use previous dialogues as context                    context = " ".join(context_buffer[-3:]) if context_buffer else ""                                        turns.append(DialogueTurn(                        context=context,                        response=dialogue,                        metadata={                            "format": "dialogue",                            "speaker": speaker.strip()                        }                    ))                                        context_buffer.append(f"{speaker}: {dialogue}")                return turns        def _clean_text(self, text: str) -> str:        """Clean and normalize text"""        text = re.sub(r'\s+', ' ', text)  # Normalize whitespace        text = text.replace('\\', ' ')     # Remove backslashes        text = text.strip()        return text        def parse_directory(self, directory: str, pattern: str = "*.txt",                        auto_save_txt: bool = True,                        output_file: str = "cleandata.txt") -> List[DialogueTurn]:        """        Parse all dialogue files in a directory.                Args:            directory: Path to directory containing dialogue files            pattern: File pattern to match (default: "*.txt")            auto_save_txt: If True, automatically saves to cleandata.txt            output_file: Name of output file (default: "cleandata.txt")                    Returns:            List of DialogueTurn objects        """        all_turns = []        file_pattern = os.path.join(directory, pattern)        files = glob.glob(file_pattern)                print(f"📁 Found {len(files)} files matching pattern: {pattern}")                # Parse each file        for i, filepath in enumerate(sorted(files), 1):            try:                turns = self.parse_file(filepath)                all_turns.extend(turns)                if i % 100 == 0:                    print(f"   Processed {i}/{len(files)} files...")            except Exception as e:                print(f"⚠️  Error parsing {os.path.basename(filepath)}: {e}")                continue                print(f"\n✅ Total dialogue turns extracted: {len(all_turns)}")                # Auto-save to cleandata.txt if enabled        if auto_save_txt and all_turns:            self.save_to_cleandata(all_turns, output_file)                return all_turns        def save_to_cleandata(self, turns: List[DialogueTurn], output_file: str = "cleandata.txt"):        """        Save parsed dialogues to cleandata.txt in 'Character: dialogue' format.        This file is used by FlexibleDialogueDataProcessor.        """        print(f"\n{'='*70}")        print(f"💾 Creating {output_file}")        print('='*70)                # Convert to dialogue format        dialogue_text = self.to_dialogue_format(turns)                # Save to file        with open(output_file, 'w', encoding='utf-8') as f:            f.write(dialogue_text)                # Calculate statistics        lines = dialogue_text.split('\n')        char_counts = {}        for line in lines:            if ':' in line:                char = line.split(':', 1)[0].strip()                char_counts[char] = char_counts.get(char, 0) + 1                print(f"✅ Saved to: {output_file}")        print(f"   Format: Character: dialogue text")        print(f"   Total lines: {len(lines)}")        print(f"   Unique characters: {len(char_counts)}")                if char_counts:            print(f"\n   Top 5 characters:")            for char, count in sorted(char_counts.items(), key=lambda x: x[1], reverse=True)[:5]:                print(f"     • {char}: {count} lines")                print('='*70)        print(f"✅ {output_file} is ready for processing!")        print('='*70)        def to_dialogue_format(self, turns: List[DialogueTurn]) -> str:        """Convert DialogueTurn objects to 'Character: dialogue' format"""        lines = []        for turn in turns:            speaker = "Unknown"            if turn.metadata:                if 'speaker' in turn.metadata:                    speaker = turn.metadata['speaker']                elif 'character' in turn.metadata:                    speaker = turn.metadata['character']                        if speaker and turn.response:                lines.append(f"{speaker}: {turn.response}")                return "\n".join(lines)print("✅ DialogueParser class defined successfully!")

In [ ]:
# ============================================================================# Parse Dialogue Files and Create cleandata.txt# ============================================================================# Initialize the dialogue parserdialogue_parser = DialogueParser()# Parse all dialogue files from the dataset directory# This automatically creates cleandata.txtturns = dialogue_parser.parse_directory(    directory=DATASET_PATH,    pattern='*.txt',    auto_save_txt=True,  # Automatically creates cleandata.txt    output_file='cleandata.txt')print(f"\n🎉 Parsing complete! Extracted {len(turns)} dialogue turns.")

## 🔤 Section 4: Data Processing & TokenizationProcess the cleandata.txt file, build vocabulary, and create training sequences.

In [ ]:
# ============================================================================# FlexibleDialogueDataProcessor Class# ============================================================================# This class processes cleandata.txt and prepares data for training:# - Loads dialogues in "Character: dialogue" format# - Detects and validates characters# - Builds vocabulary and tokenizes text# - Creates training sequences with padding# ============================================================================class FlexibleDialogueDataProcessor:    """    Processes cleandata.txt file for GAN training.    Handles character detection, tokenization, and sequence creation.    """        def __init__(self, file_path='cleandata.txt', seq_length=50,                  max_vocab_size=5000, min_char_occurrence=2):        """        Args:            file_path: Path to cleandata.txt file            seq_length: Maximum sequence length for padding            max_vocab_size: Maximum vocabulary size            min_char_occurrence: Minimum occurrences for a character to be valid        """        if not os.path.isfile(file_path):            raise ValueError(f"File not found: {file_path}")                self.path = file_path        self.seq_length = seq_length        self.max_vocab_size = max_vocab_size        self.min_char_occurrence = min_char_occurrence                # Data containers        self.dialogues = []        self.characters = defaultdict(list)        self.scenes = defaultdict(list)        self.tokenizer = None        self.vocab_size = 0        def _is_character_line(self, line):        """Check if line is in 'Character: dialogue' format"""        if ':' not in line:            return False, None, None                parts = line.split(':', 1)        if len(parts) != 2:            return False, None, None                character = parts[0].strip()        dialogue = parts[1].strip()                if not character or not dialogue:            return False, None, None                return True, character, dialogue        def load_and_parse(self, verbose=True):        """Load and parse cleandata.txt file"""        if verbose:            print(f"📖 Reading file: {self.path}")                # Read file        with open(self.path, 'r', encoding='utf-8', errors='ignore') as f:            content = f.read()                lines = content.split('\n')                # First pass: Count character occurrences        character_counts = defaultdict(int)        for line in lines:            line = line.strip()            if not line:                continue                        is_char, char_name, _ = self._is_character_line(line)            if is_char:                character_counts[char_name] += 1                # Filter characters by minimum occurrence        valid_characters = {            char for char, count in character_counts.items()            if count >= self.min_char_occurrence        }                if verbose:            print(f"\n✅ Detected {len(valid_characters)} valid characters:")            for char in sorted(valid_characters)[:10]:                print(f"   • {char}: {character_counts[char]} lines")            if len(valid_characters) > 10:                print(f"   ... and {len(valid_characters) - 10} more")                # Second pass: Parse dialogues with valid characters        current_scene = 'Unknown'                for line in lines:            line = line.strip()            if not line:                continue                        is_char, char_name, dialogue = self._is_character_line(line)                        if is_char and char_name in valid_characters:                entry = {                    'character': char_name,                    'dialogue': dialogue,                    'scene': current_scene,                }                                self.dialogues.append(entry)                self.characters[char_name].append(dialogue)                self.scenes[current_scene].append(dialogue)                if verbose:            print(f"\n✅ Loaded {len(self.dialogues)} dialogue entries")            print(f"   Characters: {len(self.characters)}")            print(f"   Unique scenes: {len(self.scenes)}")                return self.dialogues        def create_tokenizer(self):        """Create and fit tokenizer on dialogue data"""        all_dialogues = [d['dialogue'] for d in self.dialogues]                self.tokenizer = Tokenizer(            num_words=self.max_vocab_size,            filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',            lower=True,            oov_token='<UNK>'        )                self.tokenizer.fit_on_texts(all_dialogues)        self.vocab_size = min(len(self.tokenizer.word_index) + 1, self.max_vocab_size)                print(f"\n✅ Tokenizer created")        print(f"   Vocabulary size: {self.vocab_size}")        print(f"   Sample words: {list(self.tokenizer.word_index.items())[:10]}")                return self.tokenizer        def create_sequences(self, use_context=True):        """Create training sequences with sliding window"""        sequences = []        labels = []        metadata = []                for i, entry in enumerate(self.dialogues):            # Get context from previous dialogue            context = ''            if use_context and i > 0:                context = self.dialogues[i-1]['dialogue'] + ' '                        full_text = context + entry['dialogue']            token_list = self.tokenizer.texts_to_sequences([full_text])[0]                        # Create sliding windows            for j in range(1, len(token_list)):                n_gram_sequence = token_list[:j+1]                if len(n_gram_sequence) <= self.seq_length:                    sequences.append(n_gram_sequence)                    labels.append(1)  # Real data label                    metadata.append({                        'character': entry['character'],                        'scene': entry['scene']                    })                # Pad sequences        padded_sequences = pad_sequences(            sequences,            maxlen=self.seq_length,            padding='pre'        )                print(f"\n✅ Created {len(padded_sequences)} training sequences")        print(f"   Sequence shape: {padded_sequences.shape}")                return padded_sequences, np.array(labels), metadata        def get_character_encoding(self):        """Create character to index mapping"""        unique_chars = sorted(list(self.characters.keys()))        char_to_idx = {char: idx for idx, char in enumerate(unique_chars)}        idx_to_char = {idx: char for char, idx in char_to_idx.items()}        return char_to_idx, idx_to_char, len(unique_chars)print("✅ FlexibleDialogueDataProcessor class defined successfully!")

In [ ]:
# ============================================================================# Load and Process Data# ============================================================================# Initialize data processordata_processor = FlexibleDialogueDataProcessor(    file_path='cleandata.txt',    seq_length=50,    max_vocab_size=5000,    min_char_occurrence=2)# Load and parse dialoguesdialogues = data_processor.load_and_parse()# Create tokenizer and vocabularytokenizer = data_processor.create_tokenizer()vocab_size = data_processor.vocab_size# Create training sequencessequences, labels, metadata = data_processor.create_sequences(use_context=True)# Get character encodingchar_to_idx, idx_to_char, num_characters = data_processor.get_character_encoding()print(f"\n🎉 Data processing complete!")print(f"   Vocabulary size: {vocab_size}")print(f"   Training sequences: {len(sequences)}")print(f"   Number of characters: {num_characters}")

## 🧠 Section 5: Model ArchitectureDefine the Generator and Discriminator models for the GAN.

In [ ]:
# ============================================================================# Generator Model# ============================================================================# The Generator creates new dialogue sequences from random noise.# It learns to generate realistic dialogues that fool the Discriminator.# ============================================================================class Generator(Model):    """    Generator network that creates dialogue sequences.        Architecture:    - Embedding layer for tokens    - LSTM layers for sequence generation    - Dense output layer with softmax activation    """        def __init__(self, vocab_size, embedding_dim=128, lstm_units=256):        super(Generator, self).__init__()                # Layers        self.embedding = Embedding(vocab_size, embedding_dim)        self.lstm1 = LSTM(lstm_units, return_sequences=True)        self.dropout1 = Dropout(0.3)        self.lstm2 = LSTM(lstm_units, return_sequences=True)        self.dropout2 = Dropout(0.3)        self.dense = Dense(vocab_size, activation='softmax')        def call(self, inputs, training=False):        x = self.embedding(inputs)        x = self.lstm1(x)        x = self.dropout1(x, training=training)        x = self.lstm2(x)        x = self.dropout2(x, training=training)        return self.dense(x)print("✅ Generator model defined")

In [ ]:
# ============================================================================# Discriminator Model# ============================================================================# The Discriminator evaluates dialogue sequences as real or fake.# It learns to distinguish between actual dialogues and generated ones.# ============================================================================class Discriminator(Model):    """    Discriminator network that classifies dialogue sequences.        Architecture:    - Embedding layer for tokens    - Bidirectional LSTM for sequence processing    - Dense layers for classification    """        def __init__(self, vocab_size, embedding_dim=128, lstm_units=128):        super(Discriminator, self).__init__()                # Layers        self.embedding = Embedding(vocab_size, embedding_dim)        self.bilstm = Bidirectional(LSTM(lstm_units))        self.dropout = Dropout(0.5)        self.dense1 = Dense(128, activation='relu')        self.dense2 = Dense(1, activation='sigmoid')        def call(self, inputs, training=False):        x = self.embedding(inputs)        x = self.bilstm(x)        x = self.dropout(x, training=training)        x = self.dense1(x)        return self.dense2(x)print("✅ Discriminator model defined")

In [ ]:
# ============================================================================# Initialize Models# ============================================================================# Create model instancesgenerator = Generator(    vocab_size=vocab_size,    embedding_dim=128,    lstm_units=256)discriminator = Discriminator(    vocab_size=vocab_size,    embedding_dim=128,    lstm_units=128)# Compile modelsgenerator.compile(    optimizer=Adam(learning_rate=0.001),    loss='sparse_categorical_crossentropy')discriminator.compile(    optimizer=Adam(learning_rate=0.0001),    loss='binary_crossentropy',    metrics=['accuracy'])print("✅ Models initialized and compiled")print(f"   Generator parameters: {generator.count_params():,}")print(f"   Discriminator parameters: {discriminator.count_params():,}")

## 🏋️ Section 6: Model TrainingTrain the GAN with adversarial training loop.

In [ ]:
# ============================================================================# Training Configuration# ============================================================================# Training hyperparametersBATCH_SIZE = 64EPOCHS = 10DISCRIMINATOR_UPDATES = 1  # Updates per batchGENERATOR_UPDATES = 1      # Updates per batchprint("📊 Training Configuration:")print(f"   Batch size: {BATCH_SIZE}")print(f"   Epochs: {EPOCHS}")print(f"   Training sequences: {len(sequences)}")print(f"   Batches per epoch: {len(sequences) // BATCH_SIZE}")

In [ ]:
# ============================================================================# Adversarial Training Loop# ============================================================================def train_gan(generator, discriminator, sequences, epochs, batch_size):    """    Train the GAN using adversarial training.        Steps per batch:    1. Train Discriminator on real and generated sequences    2. Train Generator to fool the Discriminator    """        history = {'d_loss': [], 'g_loss': [], 'd_acc': []}        for epoch in range(epochs):        print(f"\n{'='*70}")        print(f"Epoch {epoch + 1}/{epochs}")        print('='*70)                # Shuffle data        indices = np.random.permutation(len(sequences))        sequences_shuffled = sequences[indices]                epoch_d_loss = []        epoch_g_loss = []        epoch_d_acc = []                # Training batches        num_batches = len(sequences) // batch_size                for batch in range(num_batches):            # Get real batch            start_idx = batch * batch_size            end_idx = start_idx + batch_size            real_sequences = sequences_shuffled[start_idx:end_idx]                        # ========================================            # Train Discriminator            # ========================================                        # Real sequences (label = 1)            real_labels = np.ones((batch_size, 1))                        # Generate fake sequences (label = 0)            noise = np.random.randint(0, vocab_size, (batch_size, 50))            fake_sequences = generator.predict(noise, verbose=0)            fake_sequences = np.argmax(fake_sequences, axis=-1)            fake_labels = np.zeros((batch_size, 1))                        # Train discriminator on real and fake            d_loss_real = discriminator.train_on_batch(real_sequences, real_labels)            d_loss_fake = discriminator.train_on_batch(fake_sequences, fake_labels)            d_loss = 0.5 * (d_loss_real[0] + d_loss_fake[0])            d_acc = 0.5 * (d_loss_real[1] + d_loss_fake[1])                        # ========================================            # Train Generator            # ========================================                        # Train generator to fool discriminator (label = 1)            noise = np.random.randint(0, vocab_size, (batch_size, 50))            misleading_labels = np.ones((batch_size, 1))                        # We train via discriminator but only update generator weights            discriminator.trainable = False            g_loss = discriminator.train_on_batch(                generator.predict(noise, verbose=0).argmax(axis=-1),                misleading_labels            )[0]            discriminator.trainable = True                        # Store metrics            epoch_d_loss.append(d_loss)            epoch_g_loss.append(g_loss)            epoch_d_acc.append(d_acc)                        # Print progress            if (batch + 1) % 50 == 0:                print(f"   Batch {batch + 1}/{num_batches} - "                      f"D Loss: {d_loss:.4f}, G Loss: {g_loss:.4f}, "                      f"D Acc: {d_acc:.4f}")                # Epoch summary        avg_d_loss = np.mean(epoch_d_loss)        avg_g_loss = np.mean(epoch_g_loss)        avg_d_acc = np.mean(epoch_d_acc)                history['d_loss'].append(avg_d_loss)        history['g_loss'].append(avg_g_loss)        history['d_acc'].append(avg_d_acc)                print(f"\n📊 Epoch {epoch + 1} Summary:")        print(f"   Discriminator Loss: {avg_d_loss:.4f}")        print(f"   Generator Loss: {avg_g_loss:.4f}")        print(f"   Discriminator Accuracy: {avg_d_acc:.4f}")        return historyprint("✅ Training function defined")

In [ ]:
# ============================================================================# Start Training# ============================================================================print("🚀 Starting GAN training...\n")# Train the GANtraining_history = train_gan(    generator=generator,    discriminator=discriminator,    sequences=sequences,    epochs=EPOCHS,    batch_size=BATCH_SIZE)print("\n🎉 Training complete!")

## 🎭 Section 7: Dialogue GenerationGenerate new dialogues using the trained Generator.

In [ ]:
# ============================================================================# Dialogue Generation Functions# ============================================================================def generate_dialogue(generator, tokenizer, seed_text="", max_length=50, temperature=1.0):    """    Generate dialogue sequence using the trained generator.        Args:        generator: Trained generator model        tokenizer: Tokenizer for text conversion        seed_text: Optional starting text        max_length: Maximum length of generated sequence        temperature: Sampling temperature (higher = more random)            Returns:        Generated dialogue text    """    if seed_text:        # Use seed text as starting point        tokens = tokenizer.texts_to_sequences([seed_text])[0]    else:        # Start with random token        tokens = [np.random.randint(1, tokenizer.num_words)]        # Generate tokens one by one    for _ in range(max_length - len(tokens)):        # Pad sequence        padded = pad_sequences([tokens], maxlen=50, padding='pre')                # Predict next token        predictions = generator.predict(padded, verbose=0)[0, -1]                # Apply temperature        predictions = np.log(predictions + 1e-10) / temperature        predictions = np.exp(predictions) / np.sum(np.exp(predictions))                # Sample from distribution        next_token = np.random.choice(len(predictions), p=predictions)                # Stop if end token or padding        if next_token == 0:            break                tokens.append(next_token)        # Convert tokens to text    generated_text = tokenizer.sequences_to_texts([tokens])[0]    return generated_textdef generate_character_dialogue(character_name, num_dialogues=5):    """Generate multiple dialogue lines for a specific character."""    print(f"\n🎬 Generating dialogues for: {character_name}")    print("="*70)        dialogues = []    for i in range(num_dialogues):        seed = f"{character_name}:"        dialogue = generate_dialogue(generator, tokenizer, seed_text=seed, max_length=30)        dialogues.append(dialogue)        print(f"{i+1}. {character_name}: {dialogue}")        return dialoguesprint("✅ Generation functions defined")

In [ ]:
# ============================================================================# Generate Sample Dialogues# ============================================================================print("🎭 Generating sample dialogues...\n")# Generate random dialoguesprint("Random generations:")print("="*70)for i in range(5):    dialogue = generate_dialogue(generator, tokenizer, max_length=30, temperature=0.8)    print(f"{i+1}. {dialogue}")# Generate character-specific dialogues (if characters detected)if len(char_to_idx) > 0:    sample_characters = list(char_to_idx.keys())[:3]    for character in sample_characters:        generate_character_dialogue(character, num_dialogues=3)

## 📊 Section 8: Training Visualization & EvaluationVisualize training progress and evaluate model performance.

In [ ]:
# ============================================================================# Plot Training History# ============================================================================def plot_training_history(history):    """Plot training metrics over epochs."""    fig, axes = plt.subplots(1, 3, figsize=(18, 5))        # Plot Discriminator Loss    axes[0].plot(history['d_loss'], marker='o', label='Discriminator Loss')    axes[0].set_title('Discriminator Loss', fontsize=14, fontweight='bold')    axes[0].set_xlabel('Epoch')    axes[0].set_ylabel('Loss')    axes[0].grid(True, alpha=0.3)    axes[0].legend()        # Plot Generator Loss    axes[1].plot(history['g_loss'], marker='o', color='orange', label='Generator Loss')    axes[1].set_title('Generator Loss', fontsize=14, fontweight='bold')    axes[1].set_xlabel('Epoch')    axes[1].set_ylabel('Loss')    axes[1].grid(True, alpha=0.3)    axes[1].legend()        # Plot Discriminator Accuracy    axes[2].plot(history['d_acc'], marker='o', color='green', label='Discriminator Accuracy')    axes[2].set_title('Discriminator Accuracy', fontsize=14, fontweight='bold')    axes[2].set_xlabel('Epoch')    axes[2].set_ylabel('Accuracy')    axes[2].grid(True, alpha=0.3)    axes[2].legend()        plt.tight_layout()    plt.show()# Plot the training historyplot_training_history(training_history)

In [ ]:
# ============================================================================# Model Evaluation Metrics# ============================================================================def evaluate_model(generator, discriminator, sequences, sample_size=1000):    """Evaluate model performance on sample data."""    print("\n📊 Model Evaluation")    print("="*70)        # Sample evaluation data    eval_indices = np.random.choice(len(sequences), min(sample_size, len(sequences)), replace=False)    eval_sequences = sequences[eval_indices]        # Evaluate discriminator on real sequences    real_predictions = discriminator.predict(eval_sequences, verbose=0)    real_accuracy = np.mean(real_predictions > 0.5)        # Generate fake sequences    noise = np.random.randint(0, vocab_size, (sample_size, 50))    fake_sequences = generator.predict(noise, verbose=0)    fake_sequences = np.argmax(fake_sequences, axis=-1)        # Evaluate discriminator on fake sequences    fake_predictions = discriminator.predict(fake_sequences, verbose=0)    fake_accuracy = np.mean(fake_predictions < 0.5)        # Overall metrics    overall_accuracy = (real_accuracy + fake_accuracy) / 2        print(f"✅ Discriminator Performance:")    print(f"   Real sequence detection: {real_accuracy*100:.2f}%")    print(f"   Fake sequence detection: {fake_accuracy*100:.2f}%")    print(f"   Overall accuracy: {overall_accuracy*100:.2f}%")    print("="*70)        return {        'real_accuracy': real_accuracy,        'fake_accuracy': fake_accuracy,        'overall_accuracy': overall_accuracy    }# Evaluate the modelsevaluation_metrics = evaluate_model(generator, discriminator, sequences)

## 💾 Section 9: Model PersistenceSave and load trained models for future use.

In [ ]:
# ============================================================================# Save Models# ============================================================================def save_models(generator, discriminator, tokenizer, save_dir='models'):    """Save all models and tokenizer."""    os.makedirs(save_dir, exist_ok=True)        # Save generator    generator.save(os.path.join(save_dir, 'generator.h5'))    print(f"✅ Generator saved to {save_dir}/generator.h5")        # Save discriminator    discriminator.save(os.path.join(save_dir, 'discriminator.h5'))    print(f"✅ Discriminator saved to {save_dir}/discriminator.h5")        # Save tokenizer    with open(os.path.join(save_dir, 'tokenizer.pkl'), 'wb') as f:        pickle.dump(tokenizer, f)    print(f"✅ Tokenizer saved to {save_dir}/tokenizer.pkl")        print(f"\n🎉 All models saved successfully to {save_dir}/")# Save the trained modelssave_models(generator, discriminator, tokenizer)

In [ ]:
# ============================================================================# Load Models# ============================================================================def load_models(save_dir='models'):    """Load saved models and tokenizer."""    from tensorflow import keras        # Load generator    generator = keras.models.load_model(os.path.join(save_dir, 'generator.h5'))    print(f"✅ Generator loaded from {save_dir}/generator.h5")        # Load discriminator    discriminator = keras.models.load_model(os.path.join(save_dir, 'discriminator.h5'))    print(f"✅ Discriminator loaded from {save_dir}/discriminator.h5")        # Load tokenizer    with open(os.path.join(save_dir, 'tokenizer.pkl'), 'rb') as f:        tokenizer = pickle.load(f)    print(f"✅ Tokenizer loaded from {save_dir}/tokenizer.pkl")        print(f"\n🎉 All models loaded successfully!")    return generator, discriminator, tokenizer# To load models later:# generator, discriminator, tokenizer = load_models()

## 🎯 Section 10: Summary and Next Steps### ✅ What We've Accomplished1. **Setup & Dependencies** - Installed and imported all required libraries2. **Google Drive & Data Loading** - Mounted drive and loaded dialogue datasets3. **Dialogue Parsing** - Extracted characters and dialogues from multiple text files4. **Data Processing** - Built vocabulary and created training sequences5. **Model Architecture** - Defined Generator and Discriminator networks6. **Model Training** - Trained the GAN with adversarial loss7. **Dialogue Generation** - Generated new dialogues using the trained model8. **Evaluation** - Visualized training and evaluated performance9. **Model Persistence** - Saved models for future use### 🚀 Next Steps**Improve Model Performance:**- Increase training epochs for better convergence- Experiment with different architectures (Transformer, attention mechanisms)- Add more training data for better diversity**Enhance Generation:**- Implement beam search for better quality- Add character conditioning for consistent personalities- Include context awareness for coherent dialogues**Deploy:**- Create a simple API for dialogue generation- Build a web interface for interactive generation- Integrate with chatbot frameworks### �� Additional Resources- [GAN Paper (Goodfellow et al.)](https://arxiv.org/abs/1406.2661)- [SeqGAN for Text Generation](https://arxiv.org/abs/1609.05473)- [Transformer Architecture](https://arxiv.org/abs/1706.03762)---**🎉 Congratulations! You've successfully built a GAN-based dialogue generation system!**